In [3]:
import requests
from io import StringIO
import pandas as pd
import aggregation_functions as func

On friday I decided it made more sense for the flow of the notebook to go from biosample (searching using study id) down to processed sample and then filter processed samples that cna be analyzed together using in manifest. But for some reason this traversal is giving me 32 results instead of the 33 i get when i filter by in manifest slot. I went backwards using the four outputs missing from my new traversal (but present from in manifest traversal) so i have all the ids and info, but everything looks to be correct. so i dont know hwy im missing it. 

In [4]:
stegan_biosample_info = func.get_next_results(collection='biosample_set',\
                       filter='{"associated_studies":"nmdc:sty-11-aygzgv51"}',\
                        max_page_size=100,fields='id,name,description')

In [6]:
#omics generation records for these biosamples
data_generation=func.get_id_results(newest_results=stegan_biosample_info,\
                                    id_field="id",\
                                    query_collection="data_generation_set",\
                                    match_id_field="has_input",\
                                    query_fields="has_input,has_output,associated_studies,analyte_category,type")

#data_generation=pd.DataFrame(data_generation)[pd.DataFrame(data_generation)['analyte_category']=='metaproteome'].reset_index(drop=True)
display(pd.DataFrame(data_generation))
#data_generation=data_generation.to_dict(orient='records')

#raw files produced by these omics generation records
rawfiles=func.get_id_results(newest_results=data_generation,\
                                id_field="has_output",\
                                query_collection="data_object_set",\
                                match_id_field="id",\
                                query_fields="id,data_object_type,url")

display(pd.DataFrame(rawfiles))

#analysis workflows that were run using these raw files
workflow=func.get_id_results(newest_results=rawfiles,\
                                id_field="id",\
                                query_collection="workflow_execution_set",\
                                match_id_field="has_input",\
                                query_fields="has_output,has_input")
display(pd.DataFrame(workflow))

#results from these workflows
workflow_outputs=func.get_id_results(newest_results=workflow,\
                                id_field="has_output",\
                                query_collection="data_object_set",\
                                match_id_field="id",\
                                query_fields="")

display(pd.DataFrame(workflow_outputs))

#what different data files are there for each processed data set?
pd.DataFrame(workflow_outputs)[['data_object_type','description']].value_counts()

,id,type,has_input,has_output,analyte_category,associated_studies
0,nmdc:dgms-11-00757j07,nmdc:MassSpectrometry,[nmdc:bsm-13-fs4zf767],[nmdc:dobj-11-zy24z593],lipidome,[nmdc:sty-11-aygzgv51]
1,nmdc:dgms-11-0cm4a809,nmdc:MassSpectrometry,[nmdc:bsm-13-2fw5j754],[nmdc:dobj-11-vfn59y39],lipidome,[nmdc:sty-11-aygzgv51]
2,nmdc:dgms-11-0qh41828,nmdc:MassSpectrometry,[nmdc:bsm-13-amdcp906],[nmdc:dobj-11-001b4217],lipidome,[nmdc:sty-11-aygzgv51]
3,nmdc:dgms-11-0r3k3d12,nmdc:MassSpectrometry,[nmdc:bsm-13-7qxjvr77],[nmdc:dobj-11-z2rvbs16],lipidome,[nmdc:sty-11-aygzgv51]
4,nmdc:dgms-11-0xfzga06,nmdc:MassSpectrometry,[nmdc:bsm-13-sepcxd06],[nmdc:dobj-11-s673q637],lipidome,[nmdc:sty-11-aygzgv51]
...,...,...,...,...,...,...
95,nmdc:omprc-11-r0hzrh88,nmdc:MassSpectrometry,[nmdc:bsm-13-0jw5n594],[nmdc:dobj-11-4kwfhk83],metaproteome,[nmdc:sty-11-aygzgv51]
96,nmdc:omprc-11-shwfms36,nmdc:MassSpectrometry,[nmdc:bsm-13-6naxpa52],[nmdc:dobj-11-fn371w16],metaproteome,[nmdc:sty-11-aygzgv51]
97,nmdc:omprc-11-sqhqwq89,nmdc:MassSpectrometry,[nmdc:bsm-13-c0xjg970],[nmdc:dobj-11-32dr2y07],metaproteome,[nmdc:sty-11-aygzgv51]
98,nmdc:omprc-11-w693w477,nmdc:MassSpectrometry,[nmdc:bsm-13-ay11wa90],[nmdc:dobj-11-35xc2b05],metaproteome,[nmdc:sty-11-aygzgv51]


,id,data_object_type,url
0,nmdc:dobj-11-001b4217,LC-DDA-MS/MS Raw Data,https://nmdcdemo.emsl.pnnl.gov/lipidomics/steg...
1,nmdc:dobj-11-044dk968,LC-DDA-MS/MS Raw Data,https://nmdcdemo.emsl.pnnl.gov/lipidomics/steg...
2,nmdc:dobj-11-04f4e396,LC-DDA-MS/MS Raw Data,https://nmdcdemo.emsl.pnnl.gov/lipidomics/steg...
3,nmdc:dobj-11-0tn11j64,LC-DDA-MS/MS Raw Data,https://nmdcdemo.emsl.pnnl.gov/lipidomics/steg...
4,nmdc:dobj-11-15qg0q50,LC-DDA-MS/MS Raw Data,https://nmdcdemo.emsl.pnnl.gov/lipidomics/steg...
...,...,...,...
95,nmdc:dobj-11-z65gkd65,LC-DDA-MS/MS Raw Data,https://nmdcdemo.emsl.pnnl.gov/lipidomics/steg...
96,nmdc:dobj-11-zd25ft61,LC-DDA-MS/MS Raw Data,https://nmdcdemo.emsl.pnnl.gov/proteomics/raw/...
97,nmdc:dobj-11-zv2mke38,LC-DDA-MS/MS Raw Data,https://nmdcdemo.emsl.pnnl.gov/proteomics/raw/...
98,nmdc:dobj-11-zvcn6154,LC-DDA-MS/MS Raw Data,https://nmdcdemo.emsl.pnnl.gov/lipidomics/steg...


,id,has_input,has_output
0,nmdc:wfmp-11-xdxeh614.1,"[nmdc:dobj-11-4kwfhk83, nmdc:dobj-11-fdngms90,...","[nmdc:dobj-11-pxnhdj84, nmdc:dobj-11-bm1z0e32,..."
1,nmdc:wfmp-11-2zhvtt21.1,"[nmdc:dobj-11-k9e5nm54, nmdc:dobj-11-xxffzc65,...","[nmdc:dobj-11-xm1yjv87, nmdc:dobj-11-y53nhx79,..."
2,nmdc:wfmp-11-f3tfd964.1,"[nmdc:dobj-11-xvmb4058, nmdc:dobj-11-c24s6186,...","[nmdc:dobj-11-bxyvx506, nmdc:dobj-11-m694xp08,..."
3,nmdc:wfmp-11-k5qcay19.1,"[nmdc:dobj-11-hp2vmj63, nmdc:dobj-11-p55wx268,...","[nmdc:dobj-11-z8hc0j30, nmdc:dobj-11-55vqpg72,..."
4,nmdc:wfmp-11-r1wtcg10.1,"[nmdc:dobj-11-d4qdbq55, nmdc:dobj-11-4kn1g186,...","[nmdc:dobj-11-y2aefn18, nmdc:dobj-11-znkprz15,..."
...,...,...,...
95,nmdc:wfmb-11-mdxnfx16.1,"[nmdc:dobj-11-kc99wb86, nmdc:dobj-11-e6ehsg90]","[nmdc:dobj-11-1xryts43, nmdc:dobj-11-9tcq5a73]"
96,nmdc:wfmb-11-e194xz35.1,"[nmdc:dobj-11-vw3ym510, nmdc:dobj-11-5cr3q648]","[nmdc:dobj-11-53bapp71, nmdc:dobj-11-m4ec3h74]"
97,nmdc:wfmb-11-cdy4qw71.1,"[nmdc:dobj-11-h1yd5q20, nmdc:dobj-11-jg40b630]","[nmdc:dobj-11-zr5z1921, nmdc:dobj-11-7ya2zs63]"
98,nmdc:wfmb-11-hyszqq16.1,"[nmdc:dobj-11-jgnfmx10, nmdc:dobj-11-66jz0438]","[nmdc:dobj-11-gw8dr465, nmdc:dobj-11-0mdmg029]"


,id,name,description,file_size_bytes,md5_checksum,data_object_type,url,type,in_manifest,data_category,was_generated_by
0,nmdc:dobj-11-0ysvsd12,nmdc_dobj-11-vs45wb84_nmdc_dobj-11-r3650k33_Pr...,Aggregated protein lists from MSGF+ search res...,128270,e62bc7135f22558ddf83ec1861a8febb,Protein Report,https://nmdcdemo.emsl.pnnl.gov/proteomics/resu...,nmdc:DataObject,[nmdc:manif-11-7796sg87],NaN,NaN
1,nmdc:dobj-11-1dk8mp39,nmdc_dobj-11-e2309414_nmdc_dobj-11-0f1b2t61_QC...,Overall statistics from MSGF+ search results f...,172,2f760e0bd79a7f88e9780a2596475693,Metaproteomics Workflow Statistics,https://nmdcdemo.emsl.pnnl.gov/proteomics/resu...,nmdc:DataObject,[nmdc:manif-11-7796sg87],NaN,NaN
2,nmdc:dobj-11-201gc882,nmdc_dobj-11-e2309414_nmdc_dobj-11-0f1b2t61_Pe...,Aggregated peptides sequences from MSGF+ searc...,290880,e41b272323a2a9b5c6fec2e592da8379,Peptide Report,https://nmdcdemo.emsl.pnnl.gov/proteomics/resu...,nmdc:DataObject,[nmdc:manif-11-7796sg87],NaN,NaN
3,nmdc:dobj-11-202thj43,nmdc_dobj-11-mz0d0076_nmdc_dobj-11-sf7dpr43_Pr...,Aggregated protein lists from MSGF+ search res...,60311,306a2eeedf9a2a24f26d74cc9bce31fe,Protein Report,https://nmdcdemo.emsl.pnnl.gov/proteomics/resu...,nmdc:DataObject,[nmdc:manif-11-7796sg87],NaN,NaN
4,nmdc:dobj-11-21yd6t19,nmdc_dobj-11-ms76kj12_nmdc_dobj-11-c5av0320_Pr...,Aggregated protein lists from MSGF+ search res...,41509,56e5b3fc879a0a6c295b54d87186e91c,Protein Report,https://nmdcdemo.emsl.pnnl.gov/proteomics/resu...,nmdc:DataObject,[nmdc:manif-11-7796sg87],NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
259,nmdc:dobj-11-ybs9rj39,511455_Froze_Core_2015_S1_20_30_19_Lip_POS_28J...,CoreMS hdf5 file representing a lipidomics dat...,758127644,70a3f6ea3b8a177e452364e006dea5f2,LC-MS Lipidomics Results,https://nmdcdemo.emsl.pnnl.gov/lipidomics/steg...,nmdc:DataObject,NaN,processed_data,nmdc:wfmb-11-eb4rxg76.1
260,nmdc:dobj-11-ykd98t80,511909_Froze_Core_2015_N1_50_60_11_Lip_NEG_01A...,CoreMS hdf5 file representing a lipidomics dat...,570141611,a3730c89f711ee8860e180e59181f2f9,LC-MS Lipidomics Results,https://nmdcdemo.emsl.pnnl.gov/lipidomics/steg...,nmdc:DataObject,NaN,processed_data,nmdc:wfmb-11-w1vzqx15.1
261,nmdc:dobj-11-zjeg5t88,511405_Froze_Core_2015_S3_30_40_12_Lip_POS_28J...,CoreMS hdf5 file representing a lipidomics dat...,708378192,42109b5dd0cc572a8c9c0fdc21f56416,LC-MS Lipidomics Results,https://nmdcdemo.emsl.pnnl.gov/lipidomics/steg...,nmdc:DataObject,NaN,processed_data,nmdc:wfmb-11-8b9rtn98.1
262,nmdc:dobj-11-zr5z1921,511903_Froze_Core_2015_N1_10_20_21_Lip_NEG_01A...,CoreMS hdf5 file representing a lipidomics dat...,771069220,7d9bacaa921b3f440f38b5b4e0ec6da9,LC-MS Lipidomics Results,https://nmdcdemo.emsl.pnnl.gov/lipidomics/steg...,nmdc:DataObject,NaN,processed_data,nmdc:wfmb-11-cdy4qw71.1


data_object_type                    description                                                                
LC-MS Lipidomics Results            CoreMS hdf5 file representing a lipidomics data file including annotations.    68
                                    Lipid annotations as a result of a lipidomics workflow activity.               68
Metaproteomics Workflow Statistics  Overall statistics from MSGF+ search results filtered to ~5% FDR               32
Peptide Report                      Aggregated peptides sequences from MSGF+ search results filtered to ~5% FDR    32
Protein Report                      Aggregated protein lists from MSGF+ search results filtered to ~5% FDR         32
Unfiltered Metaproteomics Results   Aggregation of analysis tools {MSGFplus, MASIC} results                        32
Name: count, dtype: int64

In [8]:
68+68+32+32+32+32

264

In [7]:
#list of workflow output ids when traversing the schema
workflow_outputs_via_traversal = pd.DataFrame(workflow_outputs)['id'].unique().tolist()

#workflow outputs when using in manifest
workflow_outputs_via_inmanifest=func.get_next_results(collection='data_object_set',\
                       filter='{"in_manifest":"nmdc:manif-11-7796sg87"}',\
                        max_page_size=100,fields='')

#workflow outputs present in in_manifest search but absent when traversing the schema
missing_entries = pd.DataFrame(workflow_outputs_via_inmanifest)[~pd.DataFrame(workflow_outputs_via_inmanifest)['id'].isin(workflow_outputs_via_traversal)]

display(missing_entries)

,id,name,description,file_size_bytes,md5_checksum,data_object_type,url,type,in_manifest
3,nmdc:dobj-11-1ty8qx90,nmdc_dobj-11-steksn39_nmdc_dobj-11-d3ahex66_QC...,Overall statistics from MSGF+ search results f...,170,0bf3449c5b84be8433680c478ea2826a,Metaproteomics Workflow Statistics,https://nmdcdemo.emsl.pnnl.gov/proteomics/resu...,nmdc:DataObject,[nmdc:manif-11-7796sg87]
15,nmdc:dobj-11-3ak2bc31,nmdc_dobj-11-steksn39_nmdc_dobj-11-d3ahex66_ms...,"Aggregation of analysis tools {MSGFplus, MASIC...",7542382,9f4e62db49b4a7fa85f1670b2207c8f4,Unfiltered Metaproteomics Results,https://nmdcdemo.emsl.pnnl.gov/proteomics/resu...,nmdc:DataObject,[nmdc:manif-11-7796sg87]
100,nmdc:dobj-11-r1en5784,nmdc_dobj-11-steksn39_nmdc_dobj-11-d3ahex66_Pe...,Aggregated peptides sequences from MSGF+ searc...,19781,ced14982cbbf6e0f170b05c842ff3e5e,Peptide Report,https://nmdcdemo.emsl.pnnl.gov/proteomics/resu...,nmdc:DataObject,[nmdc:manif-11-7796sg87]
121,nmdc:dobj-11-yddvrj28,nmdc_dobj-11-steksn39_nmdc_dobj-11-d3ahex66_Pr...,Aggregated protein lists from MSGF+ search res...,15100,452cfdb6585ac34c9cee827fc63234d0,Protein Report,https://nmdcdemo.emsl.pnnl.gov/proteomics/resu...,nmdc:DataObject,[nmdc:manif-11-7796sg87]


In [5]:
#backwards traversl using missing workflow outputs
missing_entries = missing_entries.to_dict(orient='records')

missing_workflow = func.get_id_results(newest_results=missing_entries,\
                                id_field="id",\
                                query_collection="workflow_execution_set",\
                                match_id_field="has_output",\
                                query_fields="has_output,has_input")

display(pd.DataFrame(missing_workflow))

missing_rawfiles = func.get_id_results(newest_results=missing_workflow,\
                                id_field="has_input",\
                                query_collection="data_object_set",\
                                match_id_field="id",\
                                query_fields="id,data_object_type,url")
display(pd.DataFrame(missing_rawfiles))

missing_datageneration = func.get_id_results(newest_results=missing_rawfiles,\
                                id_field="id",\
                                query_collection="data_generation_set",\
                                match_id_field="has_output",\
                                query_fields="has_input,has_output,associated_studies,analyte_category,type")
display(pd.DataFrame(missing_datageneration))


missing_biosample = func.get_id_results(newest_results=missing_datageneration,\
                                id_field="has_input",\
                                query_collection="biosample_set",\
                                match_id_field="id",\
                                query_fields="")
display(pd.DataFrame(missing_biosample))

,id,has_input,has_output
0,nmdc:wfmp-11-hpexdy53.1,"[nmdc:dobj-11-steksn39, nmdc:dobj-11-t27m4x86,...","[nmdc:dobj-11-3ak2bc31, nmdc:dobj-11-r1en5784,..."


,id,data_object_type,url
0,nmdc:dobj-11-d3ahex66,Annotation Amino Acid FASTA,https://data.microbiomedata.org/data/nmdc:ompr...
1,nmdc:dobj-11-h9637w90,NaN,https://nmdcdemo.emsl.pnnl.gov/proteomics/para...
2,nmdc:dobj-11-hfx93f93,NaN,https://nmdcdemo.emsl.pnnl.gov/proteomics/para...
3,nmdc:dobj-11-sprrem27,NaN,https://nmdcdemo.emsl.pnnl.gov/proteomics/para...
4,nmdc:dobj-11-steksn39,LC-DDA-MS/MS Raw Data,https://nmdcdemo.emsl.pnnl.gov/proteomics/raw/...
5,nmdc:dobj-11-t27m4x86,Functional Annotation GFF,https://data.microbiomedata.org/data/nmdc:ompr...


,id,has_input,has_output,type,analyte_category,associated_studies
0,nmdc:omprc-11-z841e208,[nmdc:bsm-13-7qxjvr77],[nmdc:dobj-11-steksn39],nmdc:MassSpectrometry,metaproteome,[nmdc:sty-11-aygzgv51]


,id,name,description,env_broad_scale,env_local_scale,env_medium,type,collection_date,depth,geo_loc_name,...,habitat,location,mod_date,ncbi_taxonomy_name,sample_collection_site,alternative_identifiers,insdc_biosample_identifiers,samp_name,gold_biosample_identifiers,associated_studies
0,nmdc:bsm-13-7qxjvr77,Riverbed sediment microbial communities from a...,Riverbed sediment samples were collected from ...,"{'has_raw_value': 'ENVO:01000253', 'term': {'i...","{'has_raw_value': 'ENVO:00000384', 'term': {'i...","{'has_raw_value': 'ENVO:00002007', 'term': {'i...",nmdc:Biosample,"{'has_raw_value': '2015-03-24', 'type': 'nmdc:...","{'has_raw_value': '0.0', 'has_numeric_value': ...","{'has_raw_value': 'USA: Columbia River, Washin...",...,riverbed sediment,"Columbia River, Washington, USA",2021-06-17,sediment metagenome,riverbed sediment - zone of subsurface groundw...,[img.taxon:3300042771],[biosample:SAMN06267123],GW-RW S1_0_10,[gold:Gb0126438],[nmdc:sty-11-aygzgv51]


In [6]:
display(pd.DataFrame(stegan_biosample_info)[pd.DataFrame(stegan_biosample_info)['id']=='nmdc:bsm-13-7qxjvr77'])
display(pd.DataFrame(data_generation).explode('has_input')[pd.DataFrame(data_generation).explode('has_input')['has_input']=='nmdc:bsm-13-7qxjvr77'])
display(pd.DataFrame(data_generation)[pd.DataFrame(data_generation)['id']=='nmdc:omprc-11-z841e208'])
pd.DataFrame(data_generation)

,id,name,description
30,nmdc:bsm-13-7qxjvr77,Riverbed sediment microbial communities from a...,Riverbed sediment samples were collected from ...


,id,type,has_input,has_output,analyte_category,associated_studies
3,nmdc:dgms-11-0r3k3d12,nmdc:MassSpectrometry,nmdc:bsm-13-7qxjvr77,[nmdc:dobj-11-z2rvbs16],lipidome,[nmdc:sty-11-aygzgv51]
49,nmdc:dgms-11-kvw8q644,nmdc:MassSpectrometry,nmdc:bsm-13-7qxjvr77,[nmdc:dobj-11-pfpcpm39],lipidome,[nmdc:sty-11-aygzgv51]


,id,type,has_input,has_output,analyte_category,associated_studies


,id,type,has_input,has_output,analyte_category,associated_studies
0,nmdc:dgms-11-00757j07,nmdc:MassSpectrometry,[nmdc:bsm-13-fs4zf767],[nmdc:dobj-11-zy24z593],lipidome,[nmdc:sty-11-aygzgv51]
1,nmdc:dgms-11-0cm4a809,nmdc:MassSpectrometry,[nmdc:bsm-13-2fw5j754],[nmdc:dobj-11-vfn59y39],lipidome,[nmdc:sty-11-aygzgv51]
2,nmdc:dgms-11-0qh41828,nmdc:MassSpectrometry,[nmdc:bsm-13-amdcp906],[nmdc:dobj-11-001b4217],lipidome,[nmdc:sty-11-aygzgv51]
3,nmdc:dgms-11-0r3k3d12,nmdc:MassSpectrometry,[nmdc:bsm-13-7qxjvr77],[nmdc:dobj-11-z2rvbs16],lipidome,[nmdc:sty-11-aygzgv51]
4,nmdc:dgms-11-0xfzga06,nmdc:MassSpectrometry,[nmdc:bsm-13-sepcxd06],[nmdc:dobj-11-s673q637],lipidome,[nmdc:sty-11-aygzgv51]
...,...,...,...,...,...,...
95,nmdc:omprc-11-r0hzrh88,nmdc:MassSpectrometry,[nmdc:bsm-13-0jw5n594],[nmdc:dobj-11-4kwfhk83],metaproteome,[nmdc:sty-11-aygzgv51]
96,nmdc:omprc-11-shwfms36,nmdc:MassSpectrometry,[nmdc:bsm-13-6naxpa52],[nmdc:dobj-11-fn371w16],metaproteome,[nmdc:sty-11-aygzgv51]
97,nmdc:omprc-11-sqhqwq89,nmdc:MassSpectrometry,[nmdc:bsm-13-c0xjg970],[nmdc:dobj-11-32dr2y07],metaproteome,[nmdc:sty-11-aygzgv51]
98,nmdc:omprc-11-w693w477,nmdc:MassSpectrometry,[nmdc:bsm-13-ay11wa90],[nmdc:dobj-11-35xc2b05],metaproteome,[nmdc:sty-11-aygzgv51]
